var_dose: script for obtaining data for var_dose_PLOTS.m

Author: Aparajita Budithi

 (c) Shahriyari Lab https://sites.google.com/site/leilishahriyari/
 
 part of https://github.com/ShahriyariLab/Data-driven-mathematical-model-of-FOLFIRI-treatment-for-colon-cancer
 
 If using this or related code please cite
   
   Budithi, A.; Su, S.; Kirshtein,A.; Shahriyari L.\
     Data driven mathematical model of FOLFIRI treatment for colon cancer. //Cancers.\
     (Manuscript submitted for publication).

In [ ]:
from qspmodel import *
from folfiri_qspmodel import *
import pandas as pd
import csv
import os
import scipy as sp

if not os.path.exists('Results/'):
    os.makedirs('Results/Var_doses/')
#else:
    #if not os.path.exists('Results/Var_doses/'):
        #os.makedirs('Results/Var_doses/')
        

# some global parameters
clusters=5 #number of clusters

# pre-defined parameters for treatment deltas, cell_steady, cell_extreme, drug_ratio, drug_eff, 5FU_killrate
drug_deltas=[71.3, 1.45, 2.56]
cell_steady_ind=[5, 6, 13]
cell_extreme_ind=[5, 6, 12]
#drug_levels=np.array([770, 300, 725])/cycle_time 
#drug_min=np.array([598, 208, 75])/cycle_time 
#drug_ratio=drug_levels/drug_min
drug_eff=[0.2, 0.4, 0.05]
FU_killrate=100

# create parameters for step function
# both no_cycles and cycle_time are 3 dim arrays for their respective 5fu, IR and LV values.
def infusion_par(no_cycles,cycle_time,k,j,start_treatment = 0):
    infusion_indices=np.concatenate([15*np.ones(no_cycles[0]), 16*np.ones(no_cycles[1]), 17*np.ones(no_cycles[2])]).astype(int)
    FUinfusion=np.array([2/24, 2])
    IRinfusion=np.array([0, 1/24])
    LVinfusion=np.array([1/24, 2/24])
    infusion_intervals=np.concatenate([[start_treatment+n*cycle_time[0]+FUinfusion for n in range(no_cycles[0])],
                                    [start_treatment+n*cycle_time[1]+IRinfusion for n in range(no_cycles[1])],
                                    [start_treatment+n*cycle_time[2]+LVinfusion for n in range(no_cycles[2])]])
    infusion_values=np.concatenate([(cycle_time[0]/(FUinfusion[1]-FUinfusion[0]))*drug_deltas[0]*np.ones(no_cycles[0]),
                                k*(cycle_time[1]/(IRinfusion[1]-IRinfusion[0]))*drug_deltas[1]*np.ones(no_cycles[1]),
                                j*(cycle_time[2]/(LVinfusion[1]-LVinfusion[0]))*drug_deltas[2]*np.ones(no_cycles[2])])
    
    return infusion_indices, infusion_intervals, infusion_values

#other parameters
nvar=Colon_5fu_Functions().nvar # number of variables
nparam=Colon_5fu_Functions().nparam # number of parameters

# infusion function
#r= step_vector(nvar, indices=infusion_indices, intervals=infusion_intervals, values=infusion_values)
#jump_indicator=lambda t, x: np.prod(t-infusion_intervals.flatten())


# define cell data for non-treatment parameter derivation
# rates acquired from bio research [delTn delTh delTc delTr delD delM lamgCmax lamgCmin delmu1 delmu2 delmu3 delH delIg delGb]
globvals=np.array([9.4951e-4, 0.231, 0.406, 0.231, 0.277, 0.02, 7.5e-3, 6.7152e-04, 1.07, 4.62, 58.7, 33.27, 499])
# average values of each variable across all patients [Tc mu1 Ig Gb]
meanvals=np.array([2920.305826, 132.316278, 6.603464, 20017.990343])
extremevals=np.array([2.67309712e+04, 1.23113649e+04, 1.91073391e+04, 7.21023854e+03,
                              3.41730382e+03, 4.32746936e+03, 2.31601361e+04, 3.24717521e+05,
                              1.62358760e+05, 1.05768591e+03, 1.39828734e+04, 2.46968870e+04,
                              1.02214368e+02, 1.63407223e+05])
celldata=[globvals, extremevals, meanvals]


clustercells=pd.read_csv('input/Large_Tumor_cell_data.csv').to_numpy()



#Computations of dynamics
print('Starting dynamics computations')



cons = ([0.1,0.5,1,2,5,7,10])

#initial conditions from treatment data
IC=pd.read_csv('input/5FU_LV_validation_tumor_status.csv')


patients_id = ['TCGA-CM-6172','TCGA-A6-6141','TCGA-A6-6142','TCGA-G4-6303','TCGA-A6-6781'];

#choose whether to vary 5-FU, Leucovorin or Irinotecan ('drugs')
#LV and IR = 0 when varying 5-FU
#IR = 0 and 5-FU at constant when varying LV
# 5-FU and LV kept at constant values when varying IR.
drugs = ['FU','IR','LV'] 

# solve ode for each patient
for pat in patients_id:
    
    print('Starting computations for patient '+ pat)
    # i is the patient position in dataset
    patient_pos = IC[IC.Patient_ID==pat].index.values
    i = patient_pos[0]
    tumor_status = IC.tumor_status[i]

    #create folder to store data corresponding to the patient
    folder_name = 'Patient_' + pat[-4:] + '_'+ tumor_status
    if not os.path.exists('Results/Var_doses/'+folder_name+'/'):
        os.makedirs('Results/Var_doses/'+folder_name+'/') 
        
    #time (patient 6303 dynamics take longer)
    if pat == 'TCGA-G4-6303':
        T = 6500
    else:
        T = 3500
        
    t = np.linspace(0, T, T*10 +1)
    
    for con in range(len(cons)):

        #initiate input variables
        no_cycles = np.full(3,IC.FU_number_cycles[i])
        if i != 17:
            cycle_t_FU = (IC.FU_days_to_drug_therapy_end[i]-IC.FU_days_to_drug_therapy_start[i])/no_cycles[0]
            cycle_time = np.full(3,cycle_t_FU)
        else:
            cycle_time = np.ones(3)
        drug_levels = np.zeros(3)
        drug_min = np.zeros(3)

        #customizing input variables to patient 
        cluster = IC.Cluster[i]
        drug_levels[0] = IC.FU_initial[i]/cycle_time[0]
        drug_min[0] = min(IC.FU_initial)/cycle_time[0]
        k = 0
        j = 0
        # below conditions to identify between patients taking different combinations of the drugs
        if 'IR_initial' in IC:
            no_cycles[1] = IC.IR_number_cycles[i]
            cycle_time[1] = (IC.IR_days_to_drug_therapy_end[i]-IC.IR_days_to_drug_therapy_start[i])/no_cycles[1]
            drug_levels[1]= IC.IR_initial[i]/cycle_time[1]
            drug_min[1] = min(IC.IR_initial)/cycle_time[1]
            k = 1
        else: 
            drug_levels[1] = 300/no_cycles[0]
            drug_min[1] = 208/no_cycles[0]

        if 'LV_initial' in IC:
            no_cycles[2] = IC.LV_number_cycles[i]
            cycle_time[2] = (IC.LV_days_to_drug_therapy_end[i]-IC.LV_days_to_drug_therapy_start[i])/no_cycles[2]
            drug_levels[2] = IC.LV_initial[i]/cycle_time[2]
            drug_min[2] = min(IC.LV_initial)/cycle_time[2]
            j = 1
        else: 
            drug_levels[2] = 725/no_cycles[0]
            drug_min[2] = 75/no_cycles[0]

        #### vary dose - CHOOSE which DRUG to vary from 'drugs' array
        for var_drug in ['FU','LV']:
            #var_drug = 'FU'
            ind = drugs.index(var_drug)

            if ind == 0:
            # if choosing 5-FU, keep IR and LV infusions at 0    
                k = 0
                j = 0
            elif ind == 2:
            # if choosing LV, keep IR infusion at 0
                k = 0

            drug_levels[ind] = drug_levels[ind]*cons[con]
            drug_ratio = drug_levels/drug_min    
            infusion_indices, infusion_intervals, infusion_values = infusion_par(no_cycles,cycle_time,k,j)

            #infusion func
            r= step_vector(nvar, indices=infusion_indices, intervals=infusion_intervals, values=infusion_values)
            
            #define initial conditions for patient
            IC_p = np.zeros(nvar)
            IC_p[0:14] = np.array(IC.loc[i]['Tn':'Gb'])

            #compute parameters
            QSP0=QSP.from_data([clustercells[cluster-1]]+celldata)
            qspcore=Colon_5fu_Functions(parameters=QSP0.par)
            QSP_=QSP.from_data(([drug_deltas, (clustercells[cluster-1,cell_steady_ind]/extremevals[cell_extreme_ind]), drug_ratio, drug_eff, FU_killrate]), qspcore=qspcore)
    
            print(' Parameters set. Computing the solution')
            #solve ode
            u, _ = QSP_.solve_ode(t, IC_p, 'given', inhomogeneity=r, jumps=True)
    

            wr=np.empty((t.size, nvar+1))
            wr[:,0]=t
            wr[:,1:]=u
            #scaling for drugs
            #for i in range(1,4):
                    #wr[:,-i]= wr[:,-i]*(drug_levels[-i]/drug_deltas[-i])
            c=csv.writer(open('Results/Var_doses/'+folder_name+'/'+var_drug+'-'+str(con)+'-dat.csv',"w"))  
            c.writerow(['time']+QSP_.variable_names())
            c.writerows(wr)
            del c

print('end.')